In [1]:
import torch
from typing import List, Dict, Tuple, Any
from kobert_transformers import get_tokenizer
from gluonnlp.data import SentencepieceTokenizer

/home/sung/anaconda3/envs/cose362/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = get_tokenizer()
tok = tokenizer('뽀로로는 남극에 사는 펭귄이 아니다.', padding=True, truncation=True) # Has Input_ids, token_type_ids, attention_mask
res = tokenizer.convert_ids_to_tokens(tok['input_ids'])
print(type(tok['attention_mask']))

<class 'list'>


In [3]:
label_list = ['PS', 'FD', 'TR', 'AF', 'OG', 'LC', 'CV', 'DT', 'TI', 'TI', 'QT', 'EV', 'AM', 'PT', 'MT', "TM"] 
label_fin = ['O']
label_fin += ['B-' + i for i in label_list]
label_fin += ['I-' + i for i in label_list]
label_to_idx = {label: idx for idx, label in enumerate(label_fin)}
idx_to_label = {idx: label for idx, label in enumerate(label_fin)}
print(label_to_idx, idx_to_label)

{'O': 0, 'B-PS': 1, 'B-FD': 2, 'B-TR': 3, 'B-AF': 4, 'B-OG': 5, 'B-LC': 6, 'B-CV': 7, 'B-DT': 8, 'B-TI': 10, 'B-QT': 11, 'B-EV': 12, 'B-AM': 13, 'B-PT': 14, 'B-MT': 15, 'B-TM': 16, 'I-PS': 17, 'I-FD': 18, 'I-TR': 19, 'I-AF': 20, 'I-OG': 21, 'I-LC': 22, 'I-CV': 23, 'I-DT': 24, 'I-TI': 26, 'I-QT': 27, 'I-EV': 28, 'I-AM': 29, 'I-PT': 30, 'I-MT': 31, 'I-TM': 32} {0: 'O', 1: 'B-PS', 2: 'B-FD', 3: 'B-TR', 4: 'B-AF', 5: 'B-OG', 6: 'B-LC', 7: 'B-CV', 8: 'B-DT', 9: 'B-TI', 10: 'B-TI', 11: 'B-QT', 12: 'B-EV', 13: 'B-AM', 14: 'B-PT', 15: 'B-MT', 16: 'B-TM', 17: 'I-PS', 18: 'I-FD', 19: 'I-TR', 20: 'I-AF', 21: 'I-OG', 22: 'I-LC', 23: 'I-CV', 24: 'I-DT', 25: 'I-TI', 26: 'I-TI', 27: 'I-QT', 28: 'I-EV', 29: 'I-AM', 30: 'I-PT', 31: 'I-MT', 32: 'I-TM'}


# Preprocess Part

In [4]:
'''
We will return the label of given words, using the ne_lists
We use BIO-tagging
'''
def tagging(words: List[str], ne_lists: List[Dict[str, Any]]) -> List[str] :
    results = [i if i in ['[CLS]', '[SEP]', '[PAD]'] else 'O' for i in words] # If token is not Special, initialize 'O' tag
    ps_words = [i.replace('##', '').replace('▁','') for i in words]
    ne_cnt = len(ne_lists)
    ne_idx = -1
    ne_label = 0

    for idx, word in enumerate(ps_words) :
        if results[idx] != 'O' or word == '' or word == '[UNK]':
            continue
        if word == '[UNK]' :
            continue
        # Now condition check
        if ne_idx >= 0 : 
            nw_word = ne_lists[ne_idx]['form'][ne_label:]
        else :
            nw_word = ''

        # I-tag condition
        if (len(nw_word) > 0) & (nw_word.startswith(word)) & (results[idx-1][0] == 'B' or results[idx-1][0] == 'I') :
            results[idx] = 'I-' + ne_lists[ne_idx]['label'][:2]
            ne_label += len(word)
        else : # B-tag condition
            back_idx = ne_idx
            back_label = ne_label
            while ne_idx + 1 < ne_cnt :
                ne_idx += 1
                ne_label = 0
                nw_word = ne_lists[ne_idx]['form']
                if (len(nw_word) > 0) & (nw_word.startswith(word)) :
                    results[idx] = 'B-' + ne_lists[ne_idx]['label'][:2]
                    ne_label += len(word)
                    break
            if ne_idx + 1 == ne_cnt and ne_label == 0:
                ne_idx = back_idx
                ne_label = back_label

    return results


In [5]:
sentence = "태안군의회, 2019년‘군민중심’의정성과 빛났다!"
ne = [
        {
            "id": 1,
            "form": "태안군의회",
            "label": "OGG_POLITICS",
            "begin": 0,
            "end": 5
        },
        {
            "id": 2,
            "form": "2019년",
            "label": "DT_YEAR",
            "begin": 7,
            "end": 12
        }
]

tokenizer = get_tokenizer()
tok = tokenizer(sentence, padding=True, truncation=True) # Has Input_ids, token_type_ids, attention_mask
tokens_word = tokenizer.convert_ids_to_tokens(tok['input_ids'])
print(tokens_word, tagging(tokens_word, ne), sep='\n')

['[CLS]', '▁태', '안', '군의', '회', ',', '▁20', '19', '년', '‘', '군', '민', '중심', '’', '의', '정', '성과', '▁빛', '났다', '!', '[SEP]']
['[CLS]', 'B-OG', 'I-OG', 'I-OG', 'I-OG', 'O', 'B-DT', 'I-DT', 'I-DT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '[SEP]']


# Json loads & dataframe preprocess

In [6]:
import json
import pandas as pd

def load_files(path='./dataset/NLNE2202211219.json') :
    with open(path, "r") as f :
        bef_data = json.load(f)

    bef_data = bef_data['document']

    df_tot = pd.DataFrame(columns=['form', 'NE'])

    for r in bef_data :
        df_tot = df_tot.append(pd.DataFrame.from_records(r['sentence'], columns=['form', 'NE']))

    df_tot.dropna(how='any', inplace=True)

    return df_tot

In [7]:
tt = load_files()
tt.iloc[0]

form                          태안군의회, 2019년‘군민중심’의정성과 빛났다!
NE      [{'id': 1, 'form': '태안군의회', 'label': 'OGG_POLI...
Name: 0, dtype: object

In [8]:
# def convert_df(data: List[Any]) -> pd.DataFrame:
#     return pd.DataFrame.from_records(data['sentence'], columns=['form'])
# ex = bef_data[0]
# ex = ex['sentence']
# print(type(ex), ex[1], sep='\n')
# df = pd.DataFrame.from_records(ex, columns=['form', 'NE'])
# df

In [9]:
devices = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Define DataLoader, with tokenizer
# Have to define collect_fn, to gather attention mask and another information
# tok = tokenizer('뽀로로는 남극에 사는 펭귄이 아니다.', padding=True, truncation=True) # Has Input_ids, token_type_ids, attention_mask
# tokenizer.convert_ids_to_tokens(tok['input_ids'])
df = load_files()
texts = df['form'].to_list()
ne = df['NE'].to_list()

# DataLoader

In [10]:
import os
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset) :
    def __init__(self, texts, labels, tokenizer, max_len = 256) -> None:
        self.tokenizer = tokenizer 
        self.texts = texts
        self.labels = labels
        self.max_len = max_len

    def __len__(self) :
        return len(self.texts)
    
    def __getitem__(self, index) -> Any:
        # tokenizer
        input = self.texts[index]
        sentence = self.tokenizer(input, padding = 'max_length', truncation = True, max_length = self.max_len) # Input_ids, token_type_ids, attention_mask
        tags = tagging(self.tokenizer.convert_ids_to_tokens(sentence['input_ids']), self.labels[index])
        convert_tags = [-100 if i in ['[CLS]', '[SEP]', '[PAD]'] else label_to_idx[i] for i in tags]
        return {
            'sentence' : input, # str
            'input_ids' : torch.tensor(sentence['input_ids'], dtype=torch.long).to(devices),
            'token_type_id' : torch.tensor(sentence['token_type_ids'], dtype=torch.long).to(devices),
            'attention_mask' : torch.tensor(sentence['attention_mask'], dtype=torch.long).to(devices),
            'labels' : torch.tensor(convert_tags, dtype=torch.long).to(devices)
        } # Collect_fn?

In [11]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_ne, test_ne = train_test_split(texts, ne, test_size=0.2, random_state=42)
test_dataset = CustomDataset(test_texts, test_ne, tokenizer)
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle=True)

In [37]:
cnt = 0
for batch in test_loader :
    if cnt > 1 :
        break
    cnt += 1
    print(len(batch), batch, sep='\n')

5
{'sentence': ['“위기청소년들에게 방역키트 드립니다.”', '한편 이번 임시회에서 강선구 의원이 대표 발의한 ‘예산군 긴급 생활안정 지원 조례안’과 예산군수가 제출한 ‘예산군 소상공인 지원조례 일부개정조례안’, ‘예산군 근로자 권리보호 및 증진 조례안’의 의결로 감염병 등 긴급 상황 발생으로 매출 감소, 실직, 휴직 등으로 어려움을 겪고 있는 군민과 생계에 어려움은 있으나 다른 법령에 따른 공적지원을 받지 못하는 군민의 생활안정 지원에 필요한 근거가 마련되었다.', '특히, 코로나19로 추진이 어려워진 ‘글로벌인재 해외연수’에 관해 학생들이 모두 차별없이 지속적으로 지원받을 수 있는 방향으로 대안이 마련되면 좋겠다는 것에 뜻을 모았다.', '바이오헬스는 ICT 산업 이후의 차세대 주력산업으로 꼽히지만 기본적인 용어부터가 어려운 탓에 일반인이 접근하기엔 쉽지 않은 장벽이 존재한다.', '최악의 경우, 수능 전날 진단검사를 받은 뒤 밤늦게 확진이나 자가격리 통보를 받는 수험생이 생길 수 있다.', '경남 남해군은 코로나 19로 인한 지역경제 침체를 극복하고 군민의 안전을 확보하기 위해 상반기 재난관리 및 예방사업에 대한 신속한 예산 집행으로 지역경제 활성화에 속도를 내겠다고 18일 밝혔다.', '정두수 전국가요제는 2017년까지 ‘하동 섬진강 전국가요제’로 치러졌으나 정두수 선생을 추모하고 가요제의 위상을 높여 예술의 고장 하동을 널리 알리고 참신한 신인을 발굴하고자 2018년 명칭을 바꿨다.', '제8대 대전시의회 후반기 의장 선출과 관련한 더불어민주당 내 집안싸움이 24일 4차 비공개 간담회를 통해 마침표를 찍을 전망이다.', '올해는 ▲내항동 GS 주유소부터 요암동 일원 1공구 3.2km ▲요암동부터 신흑동 흑포삼거리까지 2공구 2.5km ▲흑포삼거리부터 신흑동 일원 3공구 4km ▲신흑동 일원 1km ▲5~6공구 지역 정압기 2개 설치 등 모두 6개 공구 10.7km의 주 관로를 매설하게 되며, 이르면 연말부터 아파트 등 공동주택과 한화콘도에